### String Output Parser

In [1]:
from json import load
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

template = PromptTemplate(
    template="What is the capital of {country}?", input_variables=["country"]
)

chat = template.invoke({"country": "France"})

response = llm.invoke(chat)

print(response.content)

The capital of France is Paris.


#### using parser

In [4]:
parser = StrOutputParser()
template = PromptTemplate(
    template="What is the capital of {country}?", input_variables=["country"]
)

chat = template.invoke({"country": "France"})
result = llm.invoke(chat)
final_result = parser.invoke(result)

print(final_result)

The capital of France is Paris.


#### with chain

In [5]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
parser = StrOutputParser()
template = PromptTemplate(
    template="What is the capital of {country}?", input_variables=["country"]
)

chain = template | llm | parser
result = chain.invoke({"country": "France"})

print(result)

The capital of France is Paris.


### Json Output Parser

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

template = PromptTemplate.from_template("""
Extract the following product information from the text and return it as JSON:
- name
- price
- description
- category
- features (list)
                                        
text: {text}
format_instructions: {format_instructions}
""", 
partial_variables={"format_instructions": json_parser.get_format_instructions()}
)

chain = template | llm | json_parser

text = """
Experience the ultimate in mobile technology with the iPhone 15 Pro Max. Featuring a stunning 6.7-inch Super Retina XDR display, powerful A17
  Pro chip, and advanced Pro camera system with 48MP main camera. The aerospace-grade titanium design is both lightweight and incredibly
  durable.

  Key Features:
  - 6.7-inch Super Retina XDR display with ProMotion
  - A17 Pro chip for lightning-fast performance
  - Pro camera system with 48MP, 12MP, and 12MP lenses
  - All-day battery life with USB-C charging
  - 5G connectivity for blazing-fast downloads
  - Face ID for secure authentication
  - iOS 17 with advanced privacy features

  Price: $1,199.00
  Available in Natural Titanium, Blue Titanium, White Titanium, and Black Titanium
  Free shipping and 30-day return policy
"""

result = chain.invoke({"text": text})

print(result)

{'name': 'iPhone 15 Pro Max', 'price': '$1,199.00', 'description': 'Experience the ultimate in mobile technology with the iPhone 15 Pro Max. Featuring a stunning 6.7-inch Super Retina XDR display, powerful A17 Pro chip, and advanced Pro camera system with 48MP main camera. The aerospace-grade titanium design is both lightweight and incredibly durable.', 'category': 'Electronics', 'features': ['6.7-inch Super Retina XDR display with ProMotion', 'A17 Pro chip for lightning-fast performance', 'Pro camera system with 48MP, 12MP, and 12MP lenses', 'All-day battery life with USB-C charging', '5G connectivity for blazing-fast downloads', 'Face ID for secure authentication', 'iOS 17 with advanced privacy features']}


### Pydantic Output Parser

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Optional

class Product(BaseModel):
    name: str = Field(..., description="The name of the product")
    price: float = Field(..., description="The price of the product")
    description: str = Field(..., description="The description of the product")
    category: str = Field(..., description="The category of the product")
    features: Optional[List[str]] = Field(None, description="The features of the product")  

pydantic_parser = PydanticOutputParser(pydantic_object=Product)

template = PromptTemplate.from_template("""
Extract the following product information from the text:
                                        
text: {text}
format_instructions: {format_instructions}
""", 
partial_variables={"format_instructions": pydantic_parser.get_format_instructions()}
)

chain = template | llm | pydantic_parser

text = """
Experience the ultimate in mobile technology with the iPhone 15 Pro Max. Featuring a stunning 6.7-inch Super Retina XDR display, powerful A17
  Pro chip, and advanced Pro camera system with 48MP main camera. The aerospace-grade titanium design is both lightweight and incredibly
  durable.

  Key Features:
  - 6.7-inch Super Retina XDR display with ProMotion
  - A17 Pro chip for lightning-fast performance
  - Pro camera system with 48MP, 12MP, and 12MP lenses
  - All-day battery life with USB-C charging
  - 5G connectivity for blazing-fast downloads
  - Face ID for secure authentication
  - iOS 17 with advanced privacy features

  Price: $1,199.00
  Available in Natural Titanium, Blue Titanium, White Titanium, and Black Titanium
  Free shipping and 30-day return policy
"""

result = chain.invoke({"text": text})

print(result)

name='iPhone 15 Pro Max' price=1199.0 description='Experience the ultimate in mobile technology with the iPhone 15 Pro Max. Featuring a stunning 6.7-inch Super Retina XDR display, powerful A17 Pro chip, and advanced Pro camera system with 48MP main camera. The aerospace-grade titanium design is both lightweight and incredibly durable.' category='Electronics' features=['6.7-inch Super Retina XDR display with ProMotion', 'A17 Pro chip for lightning-fast performance', 'Pro camera system with 48MP, 12MP, and 12MP lenses', 'All-day battery life with USB-C charging', '5G connectivity for blazing-fast downloads', 'Face ID for secure authentication', 'iOS 17 with advanced privacy features']
